In [1]:
# Reference
# https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html

import pandas as pd
import requests
import pymysql
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from functions.pubgapi import get_tournament_info
from functions.pubgapi import get_match_info
from functions.pubgapi import get_match_participant_single

# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

In [3]:
# Tournament id 값 조회
tournament_info = get_tournament_info(api_key)
tournament_id = tournament_info["id"]

print(f"{len(tournament_id)} 개의 tournament_id 조회 완료\n")

# tournament_index를 지정하여 id값 출력
idx = 0
tournament_index_id = tournament_id[idx]

cur_match_info = get_match_info(api_key, tournament_index_id)
num_match = len(cur_match_info["matchId"])
print(f"Tournament name: {tournament_index_id} ({num_match} matches)")

# Match participant 조회
match_info_id = cur_match_info["matchId"]
match_info_id_single = match_info_id[0]
match_participant = get_match_participant_single(api_key, match_info_id_single)
match_participant


466 개의 tournament_id 조회 완료

Tournament name: eu-trt22 (9 matches)


,player_id,team_rank,dbnos,assists,boosts,damage_dealt,headshot_kills,heals,kill_streaks,kills,longest_kill,revives,team_kills,time_survived
0,Goop,2,0,0,0,0,0,0,0,0,0,0,0,1407
1,MarcWithaC,4,0,0,0,0,0,0,0,0,0,0,0,828
2,Jamie,5,0,0,0,0,0,0,0,0,0,0,0,285
3,Gus79,3,0,0,0,0,0,0,0,0,0,0,0,1274
4,BetaFoz,1,0,0,0,0,0,0,0,0,0,0,0,1410


In [ ]:
# Load table
upload_table = 
upload_table_name = 

In [3]:
# DB와 connect
# create_engine("mysql+pymysql://아이디:"+"암호"+"@주소:포트/데이터베이스이름?charset=utf8", encoding='utf-8')
engine = create_engine("mysql+pymysql://admin:"+password+"@"+endpoint+":3306/"+dbname+"?charset=utf8", encoding="utf-8")
conn_create = engine.connect()

# AWS RDS에 tournament_info 테이블 create
upload_table.to_sql(name=f'{upload_table_name}', con=conn_create, if_exists='replace', index=False)
conn_create.close()

# DB connect
conn = pymysql.connect(host=endpoint, user=username, password=password, db=dbname, charset='utf8', port=3306)
cur = conn.cursor()

# 업로드된 테이블 확인(SQL 사용)
sql = f"SHOW TABLES LIKE '{upload_table_name}';"
cur.execute(sql)
result = cur.fetchall()
print("SHOW TABLES: \n",result)

# 내용 확인(SELECT *)
sql = "SELECT * FROM tournament_info;"
cur.execute(sql)
result1 = cur.fetchall()
result1 = pd.DataFrame(result1)
print("\ntournament_info(database):\n",result1)

tournament_info(dataframe):
              id             createdAt
0      eu-trt22  2022-04-22T15:48:35Z
1    eu-goats22  2022-04-19T17:40:55Z
2      as-apls1  2022-04-17T10:16:50Z
3      eu-esapa  2022-04-16T16:38:55Z
4       am-pcs6  2022-04-15T23:37:54Z
..          ...                   ...
461  kr-pss18s2  2018-06-17T03:40:44Z
462  ct-mpl18s1  2018-06-15T09:29:17Z
463  kr-apl18s2  2018-06-15T07:21:54Z
464  jp-pjs18s1  2018-06-14T14:22:21Z
465  kr-pwm18s1  2018-06-13T15:38:29Z

[466 rows x 2 columns]


SHOW TABLES: 
 (('tournament_info',),) 



tournament_info(database):
               0                     1
0      eu-trt22  2022-04-22T15:48:35Z
1    eu-goats22  2022-04-19T17:40:55Z
2      as-apls1  2022-04-17T10:16:50Z
3      eu-esapa  2022-04-16T16:38:55Z
4       am-pcs6  2022-04-15T23:37:54Z
..          ...                   ...
461  kr-pss18s2  2018-06-17T03:40:44Z
462  ct-mpl18s1  2018-06-15T09:29:17Z
463  kr-apl18s2  2018-06-15T07:21:54Z
464  jp-pjs18s1  2018-06-14T14:22:21Z


In [4]:
# Save tournament_info.csv
tournament_info.to_csv("./Data/tournament_info.csv")

conn.commit()

# db접속 종료
conn.close()